# ELMo

Note that you will need to use the non-GPU accelerated run-time on this notebook due to the large memory useage of the ELMo model.

## Imports:

In [0]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing

import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en')
from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

## Get the data 

The below downloads a Pandas Dataframe which is publically hosted on Google Drive (this should therefore work for anyone)

In [31]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


file_id = '1M_XljfV5t_nGjvhyfTPO9n2nfOweMwYx'
destination = 'temp'
download_file_from_google_drive(file_id, destination)

combined = pd.read_pickle('temp')

combined.head()

,Company,URL,Industry,HQ,Also Covers Companies,UK Modern Slavery Act,California Transparency in Supply Chains Act,Period Covered,text,pdf,error,FT_tfidf
0,118 118 Money,https://www.118118money.com/anti-slavery-state...,Consumer Finance,United Kingdom,NaN,True,False,2016-2017,Introduction\n\nThis statement is made pursuan...,0,0,"[-0.09019677307999371, 0.23825215930123844, 0...."
1,1Spatial Plc,https://1spatial.com/who-we-are/legal/modern-s...,Software,United Kingdom,NaN,True,False,2017,While 1Spatial’s turnover is below £36m and th...,0,0,"[-0.5010607985753625, 0.42660413175930045, -0...."
2,20/20 Projects,http://www.2020projects.co.uk/modernslaverypolicy,Commercial Services & Supplies,United Kingdom,NaN,True,False,2015-2016,html error,0,1,"[0.9405487179756165, 0.40332984924316406, 0.70..."
3,2M Holdings Ltd,https://www.2m-holdings.com/2m-holdings-ltd-mo...,Distributors,United Kingdom,NaN,True,False,2015-2016,Modern slavery is a crime resulting in an abho...,0,0,"[-0.390252637821283, 0.488747594191321, -0.238..."
4,3i Group plc,https://www.3i.com/media/3815/modern-slavery-s...,Capital Markets,United Kingdom,NaN,True,False,2017-2018,pdf error tika,1,1,"[1.0879868624172204, 0.44540999591903685, 0.80..."


## Create sentence embeddings

In [0]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

In [32]:
combined.loc[combined.Company.str.contains("Asos")]

,Company,URL,Industry,HQ,Also Covers Companies,UK Modern Slavery Act,California Transparency in Supply Chains Act,Period Covered,text,pdf,error,FT_tfidf
494,Asos plc,https://www.asosplc.com/~/media/Files/A/Asos-V...,Internet & Direct Marketing Retail,United Kingdom,NaN,True,False,2016-2018,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,0,"[-0.8573993510860287, 0.11926992131730585, -0...."
495,Asos plc,https://www.asosplc.com/~/media/Files/A/ASOS/r...,Internet & Direct Marketing Retail,United Kingdom,NaN,True,False,2015-2016,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1,0,"[-0.6513810935113411, 0.04600498146602333, 0.0..."


In [3]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg



[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [50]:
from string import punctuation

bible_kjv_sents = gutenberg.sents('bible-kjv.txt') 
preprocessed = [[word.lower() for word in sent] for sent in bible_kjv_sents]

[print(preprocessed[i]) for i in range(10)]
print("\n No. of sentences: ", len(preprocessed))

['[', 'the', 'king', 'james', 'bible', ']']
['the', 'old', 'testament', 'of', 'the', 'king', 'james', 'bible']
['the', 'first', 'book', 'of', 'moses', ':', 'called', 'genesis']
['1', ':', '1', 'in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth', '.']
['1', ':', '2', 'and', 'the', 'earth', 'was', 'without', 'form', ',', 'and', 'void', ';', 'and', 'darkness', 'was', 'upon', 'the', 'face', 'of', 'the', 'deep', '.']
['and', 'the', 'spirit', 'of', 'god', 'moved', 'upon', 'the', 'face', 'of', 'the', 'waters', '.']
['1', ':', '3', 'and', 'god', 'said', ',', 'let', 'there', 'be', 'light', ':', 'and', 'there', 'was', 'light', '.']
['1', ':', '4', 'and', 'god', 'saw', 'the', 'light', ',', 'that', 'it', 'was', 'good', ':', 'and', 'god', 'divided', 'the', 'light', 'from', 'the', 'darkness', '.']
['1', ':', '5', 'and', 'god', 'called', 'the', 'light', 'day', ',', 'and', 'the', 'darkness', 'he', 'called', 'night', '.']
['and', 'the', 'evening', 'and', 'the', 'morning',

In [4]:
bible_kjv = gutenberg.raw('bible-kjv.txt') 

bible_kjv[1:1000]

'The King James Bible]\n\nThe Old Testament of the King James Bible\n\nThe First Book of Moses:  Called Genesis\n\n\n1:1 In the beginning God created the heaven and the earth.\n\n1:2 And the earth was without form, and void; and darkness was upon\nthe face of the deep. And the Spirit of God moved upon the face of the\nwaters.\n\n1:3 And God said, Let there be light: and there was light.\n\n1:4 And God saw the light, that it was good: and God divided the light\nfrom the darkness.\n\n1:5 And God called the light Day, and the darkness he called Night.\nAnd the evening and the morning were the first day.\n\n1:6 And God said, Let there be a firmament in the midst of the waters,\nand let it divide the waters from the waters.\n\n1:7 And God made the firmament, and divided the waters which were\nunder the firmament from the waters which were above the firmament:\nand it was so.\n\n1:8 And God called the firmament Heaven. And the evening and the\nmorning were the second day.\n\n1:9 And God said

In [33]:
text = combined.iloc[494].text
text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nModern Slavery Statement \nSeptember 2016 - March 2018\n\nFACTS & FIGURES - 31 AUGUST 2017\n\nSlavery, servitude, forced labour, bonded labour, and human trafficking are issues of increasing global concern, affecting all sectors, \nregions and economies. Modern slavery is fundamentally unacceptable within our business and supply chain, and combatting it is an \n\nimportant element of our overall approach to business and human rights. ASOS is committed to respecting, protecting and championing \nthe human rights of all those who come into contact with our operations, including employees, supply chain workers, customers and \n\nlocal communities.  We accept our responsibility to support transparency; to find and resolve problems, to regularly review our business \npractices, and to collaborate with others to protect the rights of workers, particularly those who are most vulnerable to abuses such as \

In [5]:
text = gutenberg.raw('bible-kjv.txt')
import re

text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
text = ' '.join(text.split())

text

Output hidden; open in https://colab.research.google.com to view.

In [6]:
nlp.max_length = len(text)
doc = nlp(text)

sentences = []
for i in doc.sents:
  if len(i)>1:
    sentences.append(i.string.strip())
    
len(sentences)

35946

In [38]:
type(sentences)

list

In [7]:
sentences[0:10]

['[the king james bible] the old testament of the king james bible the first book of moses:',
 'called genesis 1:1 in the beginning god created the heaven and the earth.',
 '1:2 and the earth was without form, and void; and darkness was upon the face of the deep.',
 'and the spirit of god moved upon the face of the waters.',
 '1:3 and god said, let there be light: and there was light.',
 '1:4 and god saw the light, that it was good: and god divided the light from the darkness.',
 '1:5 and god called the light day, and the darkness he called night.',
 'and the evening and the morning were the first day.',
 '1:6 and',
 'god said, let there be a firmament in the midst of the waters, and let it divide the waters from the waters.']

In [0]:
embeddings = embed(
    sentences,
    signature="default",
    as_dict=True)["default"]

In [0]:
%%time
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  x = sess.run(embeddings)

In [13]:
!pip install chart-studio

     |████████████████████████████████| 81kB 3.9MB/s 


## Visualize the sentences using PCA and TSNE

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
y = pca.fit_transform(x)

from sklearn.manifold import TSNE

y = TSNE(n_components=2).fit_transform(y)

In [16]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


data = [
    go.Scatter(
        x=[i[0] for i in y],
        y=[i[1] for i in y],
        mode='markers',
        text=[i for i in sentences],
    marker=dict(
        size=16,
        color = [len(i) for i in sentences], #set color equal to a variable
        opacity= 0.8,
        colorscale='Viridis',
        showscale=False
    )
    )
]
layout = go.Layout()
layout = dict(
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
fig = go.Figure(data=data, layout=layout)
file = plot(fig, filename='Sentence encode.html')

from google.colab import files
files.download('Sentence encode.html') 

## Create a semantic search engine:

In [20]:
#@title Sementic search
#@markdown Enter a set of words to find matching sentences. 'results_returned' can beused to modify the number of matching sentences retured. To view the code behind this cell, use the menu in the top right to unhide...
search_string = "phone" #@param {type:"string"}
results_returned = "3" #@param [1, 2, 3]

from sklearn.metrics.pairwise import cosine_similarity


embeddings2 = embed(
    [search_string],
    signature="default",
    as_dict=True)["default"]

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  search_vect = sess.run(embeddings2)
  

cosine_similarities = pd.Series(cosine_similarity(search_vect, x).flatten())
output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
  output +='<p style="font-family:verdana; font-size:110%;"> '
  for i in sentences[i].split():
    if i.lower() in search_string:
      output += " <b>"+str(i)+"</b>"
    else:
      output += " "+str(i)
  output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))
#   print(sentences[i])
#   print('\n')
